## Download y grabado en csv de los datos de multas del Ayutamiento Madrid

Vamos a analizar el fichero de multas del **Ayuntamiento de Madrid**, con información sacada del portal de OPenData : http://datos.madrid.es

Como siempre importamos las librerias necesarias : pandas, numpy y matplotlib

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import datetime

In [4]:
pd.options.display.float_format = '{:,.1f}'.format

Preparamos una texto para incluirlo en cada gráfico como fuente...

In [5]:
fuente='Fuente : Ayuntamiento de Madrid, http://datos.madrid.es'

Preparando la URL de la fuente de datos

In [6]:
url_fichero = 'https://datos.madrid.es/egob/catalogo/210104-{}-multas-circulacion-detalle.csv'

cabecera de las columnas

In [7]:
nombre_columnas=['CALIFICACION','LUGAR','MES','ANIO','HORA','IMP_BOL','DESCUENTO','PUNTOS','DENUNCIANTE','HECHO_BOL','VEL_LIMITE','VEL_CIRCULA','COORDENADA_X','COORDENADA_Y']

Leemos los datos desde su localizacion en 'path_web', en este fichero tenemos los datos de Junio de 2017. Al respecto de la la identificacion de la multa en el tiempo tendremos la hora pero no el día del mes, es decir : tendremos las multas puestas a las 13:10 a lo largo de todo el mes, pero no podremos partirlas por día. No encuentro otra razón que no sea evitar cualquier vía de identificación del conductor.

El código 226 corresponde a Enero19, bajamos hasta hoy..

In [62]:
codigos_ficheros = list(range(226,247,4)) + list(range(252,265,4)) + list(range(266,297,2))

In [63]:
multas=pd.DataFrame()
for codigo in codigos_ficheros:
    path = url_fichero.format(str(codigo))
    print (path)
    req = Request(path)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    content = urlopen(req)
    multas_mes=pd.read_csv(content,sep=";",encoding='windows-1250',index_col=False,header=None,skiprows=1,low_memory=False)
    multas = pd.concat([multas, multas_mes], ignore_index=True, sort=False)

https://datos.madrid.es/egob/catalogo/210104-226-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-230-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-234-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-238-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-242-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-246-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-252-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-256-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-260-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-264-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-266-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/210104-268-multas-circulacion-detalle.csv
https://datos.madrid.es/egob/catalogo/21

In [64]:
multas.columns = nombre_columnas

In [65]:
multas.shape

(4951098, 14)

In [66]:
multas.to_csv('historico_multas.csv')

confirmamos que ha bajado correctamente

In [35]:
multas = pd.read_csv('historico_multas.csv')

In [36]:
multas['FECHA']=multas[['ANIO','MES']].astype(str).apply('-'.join,1)

In [38]:
multas['FECHA']=pd.to_datetime(multas['FECHA'],format= '%Y-%m')

In [39]:
multas = multas.drop(columns = ['MES','ANIO'])

In [40]:
multas['HORA'] = pd.to_datetime(multas['HORA'].astype(str),format='%H.%M').dt.time

In [41]:
multas[['VEL_CIRCULA','VEL_LIMITE']] = multas[['VEL_CIRCULA','VEL_LIMITE']].astype(float,errors='ignore')


In [42]:
multas.columns

Index(['CALIFICACION', 'LUGAR', 'HORA', 'IMP_BOL', 'DESCUENTO', 'PUNTOS',
       'DENUNCIANTE', 'HECHO_BOL', 'VEL_LIMITE', 'VEL_CIRCULA', 'COORDENADA_X',
       'COORDENADA_Y', 'FECHA'],
      dtype='object')

In [43]:
multas = multas[['CALIFICACION', 'LUGAR', 'FECHA','HORA', 'IMP_BOL', 'DESCUENTO', 'PUNTOS',
       'DENUNCIANTE', 'HECHO_BOL', 'VEL_LIMITE', 'VEL_CIRCULA', 'COORDENADA_X',
       'COORDENADA_Y']]

In [45]:
multas

,CALIFICACION,LUGAR,FECHA,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO_BOL,VEL_LIMITE,VEL_CIRCULA,COORDENADA_X,COORDENADA_Y
0,LEVE,SAN RESTITUTO 24,2019-01-01,16:12:00,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO. ...,,,,
1,LEVE,LOMBIA-JORGE JUAN,2019-01-01,17:35:00,60.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR MOTOCICLETA O CICLOMOTOR SOBRE LA A...,,,,
2,LEVE,CL MULLER 36,2019-01-01,10:47:00,90.0,SI,0,SER,ESTACIONAR SOBRE LA ACERA. ...,,,,
3,LEVE,SANTA ISABEL 31,2019-01-01,01:15:00,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR RESERVADO TEMPORALMENTE PA...,,,,
4,GRAVE,CAUNEDO 27,2019-01-01,09:04:00,200.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN ZONA SEŃALIZADA PARA USO EXCLUSI...,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951093,LEVE,MANZANARES 2 E,2021-02-01,12:44:00,60.0,SI,0,AGENTES DE MOVILIDAD,ESTACIONAR MOTOCICLETA O CICLOMOTOR SOBRE LA A...,,,,
4951094,GRAVE,PO RECOLETOS 1,2021-02-01,01:35:00,200.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN ESPACIOS EXPRESAMENTE RESERVADOS...,,,,
4951095,LEVE,CL DIVINO VALLES 1,2021-02-01,10:08:00,60.0,SI,0,SER,ESTACIONAR CON AUTORIZACIÓN NO VÁLIDA. ...,,,,
4951096,LEVE,GENERAL DIAZ PORLIER 78,2021-02-01,03:03:00,90.0,SI,0,POLICIA MUNICIPAL,ESTACIONAR EN LUGAR PROHIBIDO. ...,,,,


In [46]:
multas.to_csv('historico_multas.csv',index_label=False)

In [246]:
multas.shape

(4545057, 18)